In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from pathlib import Path
import yaml

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from assistive_arm.utils.plotting import extract_muscle_activations, plot_res_assist_forces, plot_residual_forces

### Setup paths

In [ ]:
# plt.style.use('fivethirtyeight')
plt.style.use('../paper.mplstyle')

In [ ]:
solutions_path = Path("../moco/control_solutions/")
subject = "subject_4"
subject_data = Path("/Users/xabieririzar/Desktop/Life/Studium/TUM/Master_Robotics/Harvard/Thesis/subject_testing/subject_data/") / subject
trial = subject_data / "trial_4"

# solution_assist_false = solutions_path / "opencap_simple_assistance_false_2023-09-22_17-13.sto"
solution_assist_false = solutions_path / "subject_4_simple_trial_3_assistance_none_2023-11-15_14-59.sto"
solution_assist_true = solutions_path / "subject_4_simple_trial_4_assistance_true_2023-10-11_08-20.sto"
grfs = trial / "grf_filtered.mot"

### Read control solutions and config file

In [ ]:
df_assist_false = pd.read_csv(solution_assist_false, delimiter="\t", skiprows=18)
df_assist_true = pd.read_csv(solution_assist_true, delimiter="\t", skiprows=18)
ground_forces = pd.read_csv(grfs, delimiter="\t", skiprows=6)

config_path = solutions_path / f"{solution_assist_false.stem}.yaml"
with open(config_path, "r") as f:
    config_file = yaml.load(f, Loader=yaml.FullLoader)
print(yaml.dump(config_file, indent=4, sort_keys=False))

In [ ]:
# def plot_residual_forces(df: pd.DataFrame, config_file: dict):
#     tx = 'reserve_pelvis_tx' if config_file["minimal_actuators"] else '/forceset/reserve_jointset_ground_pelvis_pelvis_tx'
#     ty = 'reserve_pelvis_ty' if config_file["minimal_actuators"] else '/forceset/reserve_jointset_ground_pelvis_pelvis_ty'
#     tz = 'reserve_pelvis_tz' if config_file["minimal_actuators"] else '/forceset/reserve_jointset_ground_pelvis_pelvis_tz'

#     plt.plot(df.time, df[tx]*config_file["actuator_magnitude"], label=f'Residual X')
#     plt.plot(df.time, df[ty]*config_file["actuator_magnitude"], label=f'Residual Y')
#     plt.plot(df.time, df[tz]*config_file["actuator_magnitude"], label=f'Residual Z')
#     plt.xlabel("Time [s]")
#     plt.ylabel("Force [N]")
#     plt.title(f"Residual forces ({'assisted' if config_file['assistive_force'] else 'unassisted'})")
#     plt.legend()
#     plt.show()

In [ ]:
# plot_residual_forces(df=df_assist_false, config_file=config_file)

In [ ]:
ground_forces = ground_forces[(ground_forces["time"] >= config_file["t_0"]) & (ground_forces["time"] <= config_file["t_f"])]

dfs = {}
# dfs["assist_false"] = df_assist_false
dfs["assist_true"] = df_assist_true
dfs["ground_forces"] = ground_forces

In [ ]:
# plot_res_assist_forces(time=df_assist_true.time, dataframes=dfs, figsize=(8,5), config=config_file)

### Plot residual forces

In [ ]:
# plot_residual_forces(df_assist_true, config_file=config_file)

In [ ]:
simple_df_false_activations = extract_muscle_activations(df_assist_false, apply_filter=False)
simple_df_true_reduced_activations = extract_muscle_activations(df_assist_true, apply_filter=False)[simple_df_false_activations.columns]

# Only look at recfem and vasmed
simple_df_false_activations = simple_df_false_activations[["recfem_r", "vasmed_r", "bflh_r", "bfsh_r", "recfem_l", "vasmed_l", "bflh_l", "bfsh_l", "soleus_r", "soleus_l", "tibant_r", "tibant_l"]]
simple_df_false_activations = simple_df_false_activations[["recfem_r", "vasmed_r", "recfem_l", "vasmed_l", "soleus_r", "soleus_l", "bflh_r", "bflh_l", "tibant_r", "tibant_l"]]
simple_df_true_reduced_activations = simple_df_true_reduced_activations[["recfem_r", "vasmed_r", "bflh_r", "bfsh_r", "recfem_l", "vasmed_l", "bflh_l", "bfsh_l"]]#, "vasmed_l", "soleus_r", "soleus_l", "tibant_r", "tibant_l"]]

In [ ]:
fig, axs = plt.subplots(figsize=(8, 4), sharex=True)

fig.suptitle("STS OpenSim activations")

# Activations with no assistance
axs.plot(df_assist_false.time, simple_df_false_activations, label=simple_df_false_activations.columns)
axs.set_xlabel("Time (s)")
axs.set_ylabel("Activation (%)") # percentage
axs.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, symbol=None))

handles, labels = axs.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.93),
           ncol=5, fontsize=10)
# Define xlabel

plt.tight_layout(rect=[0, 0.03, 1, 0.93])
plt.savefig("/Users/xabieririzar/Desktop/Life/Studium/TUM/Master_Robotics/Harvard/Thesis/Thesis LaTeX/Master Thesis Xabier Irizar/figures/emg_plots/osim_activations.png", dpi=500, bbox_inches='tight')
plt.show()